# Analysis of deep mutational scanning of the hemagglutinin of WSN influenza virus by Doud and Bloom (2016)

## Overview
[Doud and Bloom (2016)](http://www.mdpi.com/1999-4915/8/6/155) performed deep mutational scanning on the hemagglutinin (HA) from A/WSN/1933 (H1N1) influenza virus.
In that original paper, the data was analyzed using the older [dms_tools](https://github.com/jbloomlab/dms_tools) software package.
Here we re-analyze the data using the newer [dms_tools2](https://github.com/jbloomlab/dms_tools2) software.

## Experimental summary
The goal of the experiment was to quantify the effect of all amino-acid mutations to HA on viral replication in cell culture.
To do this, [Doud and Bloom (2016)](http://www.mdpi.com/1999-4915/8/6/155) created three independent plasmid mutant libraries carrying nearly all codon mutations of HA. 
These plasmid mutant libraries are referred to as *mutDNA-1*, *mutDNA-2*, and *mutDNA-3*.
They then used these libraries to generate three libraries of mutant viruses, which they passaged at low MOI to create a genotype-phenotype link.
These three virus mutant libraries are referred to as *mutvirus-1*, *mutvirus-2*, and *mutvirus-3*.
In addition, they performed a single replicate of the some procedure using **wildtype** (unmutated plasmid) to create non-mutagenized virus -- these plasmid and virus samples are referred to as *wtDNA* and *wtvirus*.

All of these plasmid and virus samples were deep sequenced using barcoded-subamplicon sequencing to obtain high accuracy.
This notebook analyzes those data to determine the "preference" of each site in HA for each possible amino acid.

## Import modules and define general functions / variables
Import the Python modules used by this notebook, print the version of [dms_tools2](https://github.com/jbloomlab/dms_tools2) being used, and define some general functions.

Some key variables defined below:

* **ncpus** is the number of CPUs to use. This number should not exceed the number of CPUs that you specify if you are submitting your job using some queue-ing system such as `slurm` / `sbatch`.

* **use_existing** specifies whether we use existing results if they exist, or specify everything new. Note that `dms_tools2` programs just check for existing output but do not do true dependency tracking (as would be done by `make`, for instance) -- so if you have changed key inputs then set this option to *False* to re-run everything fresh.

In [1]:
import os
from collections import OrderedDict
import pandas
from IPython.display import display, HTML
import dms_tools2

print("Using dms_tools2 version {0}".format(dms_tools2.__version__))

# results will go in this directory
resultsdir = './results/' 
if not os.path.isdir(resultsdir):
    os.mkdir(resultsdir)
    
# CPUs to use, should not exceed the number you request with slurm
ncpus = 14

# do we use existing results or generate everything new?
use_existing = True

Using dms_tools2 version 2.0.dev0


## Download the FASTQ files from the SRA
The first step is to obtain the FASTQ files with the deep sequencing data.
[Doud and Bloom (2016)](http://www.mdpi.com/1999-4915/8/6/155) have already transferred these FASTQ files from their local server to the [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra), so we simply download those files.

We download the FASQ files using the [fastq-dump](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc&f=fastq-dump) program from the [SRA Toolkit](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc).
If you do not already have this toolkit installed, you will need to install it by [following these instructions](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc&f=std).

The code cell below does the following:
1. Defines the SRA accession number for each sample.
2. Downloads each sample using `fastq-dump`. To understand the commands used by `fastq-dump` below, [look here](https://edwards.sdsu.edu/research/fastq-dump/) since the [NCBI's instructions](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc&f=fastq-dump) are sparse.
3. Renames each set of FASTQ files to have nice names like `mutDNA-1_R1.fastq.gz`.

In [2]:
# directory for FASTQ files
fastqdir = './FASTQ_files/' 
if not os.path.isdir(fastqdir):
    os.mkdir(fastqdir)

# define SRA accessions for each sample
sample_to_acc = OrderedDict([
        ('mutDNA-1', 'SRR3113656'),
        ('mutDNA-2', 'SRR3113657'),
        ('mutDNA-3', 'SRR3113658'),
        ('mutvirus-1', 'SRR3113660'),
        ('mutvirus-2', 'SRR3113661'),
        ('mutvirus-3', 'SRR3113662'),
        ('wtDNA', 'SRR3113655'),
        ('wtvirus', 'SRR3113659'),
        ])

print("Downloading FASTQ files from the SRA using:")
!fastq-dump --version

for (sample, acc) in sample_to_acc.items():
    # r1file / r2file are final names we will assign files
    r1file = '{0}/{1}_R1.fastq.gz'.format(fastqdir, sample)
    r2file = r1file.replace('R1', 'R2')
    # r1filedownload / r2filedownload are initial names after fastq-dump download
    r1filedownload = '{0}/{1}_pass_1.fastq.gz'.format(fastqdir, acc)
    r2filedownload = '{0}/{1}_pass_2.fastq.gz'.format(fastqdir, acc)
    if all(map(os.path.isfile, [r1file, r2file])):
        print("FASTQ files for {0} already exist".format(sample))
    else:
        print("Downloading FASTQ files for {0}...".format(sample))
        !fastq-dump --outdir {fastqdir} --gzip --readids --skip-technical \
                --dumpbase --clip --split-files --read-filter pass {acc}
        os.rename(r1filedownload, r1file)
        os.rename(r2filedownload, r2file)


fastq-dump : 2.8.2

FASTQ files for mutDNA-1 already exist
FASTQ files for mutDNA-2 already exist
FASTQ files for mutDNA-3 already exist
FASTQ files for mutvirus-1 already exist
FASTQ files for mutvirus-2 already exist
FASTQ files for mutvirus-3 already exist
FASTQ files for wtDNA already exist
FASTQ files for wtvirus already exist


## Align the deep sequencing data and count mutations
The alignments are made to the wildtype WSN HA coding sequence, which is available at [./data/WSN-HA.fasta](./data/WSN-HA.fasta).

In [3]:
# file containing wildtype WSN HA sequence
refseq = './data/WSN-HA.fasta'

# define subamplicon alignment specifications
alignspecs = ' '.join(['1,285,36,37', 
                       '286,570,31,32',
                       '571,855,37,32',
                       '856,1140,31,36',
                       '1141,1425,29,33',
                       '1426,1698,40,43'])

# counts and alignments placed in this directory
countsdir = os.path.join(resultsdir, 'codoncounts')
if not os.path.isdir(countsdir):
    os.mkdir(countsdir)

# countsbatch is a dataframe that defines batch input to dms2_batch_bcsubamp
countsbatch = pandas.DataFrame({'name':list(sample_to_acc.keys())})
countsbatch['R1'] = countsbatch['name'].astype('str') + '_R1.fastq.gz'
countsbatch['plotgroup'] = '' # all samples are in the same plotgroup
print("We will process the following samples:")
display(HTML(countsbatch.to_html(index=False)))
countsbatchfile = os.path.join(countsdir, 'batch.csv')
countsbatch.to_csv(countsbatchfile, index=False)

print('\nNow running dms2_batch_bcsubamp...')
!dms2_batch_bcsubamp \
        --batchfile {countsbatchfile} \
        --refseq {refseq} \
        --alignspecs {alignspecs} \
        --outdir {countsdir} \
        --summaryprefix summary \
        --R1trim 200 \
        --R2trim 170 \
        --fastqdir {fastqdir} \
        --ncpus {ncpus} \
        --use_existing

We will process the following samples:


name,R1,plotgroup
mutDNA-1,mutDNA-1_R1.fastq.gz,
mutDNA-2,mutDNA-2_R1.fastq.gz,
mutDNA-3,mutDNA-3_R1.fastq.gz,
mutvirus-1,mutvirus-1_R1.fastq.gz,
mutvirus-2,mutvirus-2_R1.fastq.gz,
mutvirus-3,mutvirus-3_R1.fastq.gz,
wtDNA,wtDNA_R1.fastq.gz,
wtvirus,wtvirus_R1.fastq.gz,



Now running dms2_batch_bcsubamp...
2017-08-09 08:14:11,247 - INFO - Beginning execution of dms2_batch_bcsubamp in directory /home/jbloom/dms_tools2/examples/Doud2016

2017-08-09 08:14:11,247 - INFO - Progress is being logged to ./results/codoncounts/summary.log
2017-08-09 08:14:11,247 - INFO - Version information:
	Time and date: Wed Aug  9 08:14:11 2017
	Platform: Linux-3.13.0-68-generic-x86_64-with-Ubuntu-14.04-trusty
	Python version: 3.4.3 (default, Oct 14 2015, 20:28:29)  [GCC 4.8.4]
	dms_tools2 version: 2.0.dev0
	Bio version: 1.68
	HTSeq version: 0.9.1
	pandas version: 0.20.3
	numpy version: 1.13.1

2017-08-09 08:14:11,247 - INFO - Parsed the following arguments:
	R1trim = [200]
	R2trim = [170]
	bclen = 8
	minreads = 2
	outdir = ./results/codoncounts
	minfraccall = 0.9
	seed = 1
	summaryprefix = summary
	use_existing = True
	bcinfo = False
	purgeread = 0
	alignspecs = ['1,285,36,37', '286,570,31,32', '571,855,37,32', '856,1140,31,36', '1141,1425,29,33', '1426,1698,40,43']
	maxmut